In [1]:
%pip install -qq "embedchain==0.0.59"

Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import Markdown

In [3]:
from embedchain import App
from embedchain.config import LlmConfig

pg_bot = App()
pg_bot.add("http://paulgraham.com/greatwork.html")

Doc content has not changed. Skipping creating chunks and embeddings


'161fcad59ba37a28e60a45f5355dab43'

In [4]:
res = pg_bot.query("how to do great work?", dry_run=True)
Markdown(res)


  Use the following pieces of context to answer the query at the end.
  If you don't know the answer, just say that you don't know, don't try to make up an answer.

  interesting, which you necessarily will if you're on the right path, the work will probably feel less burdensome than a lot of your peers'.The discoveries are out there, waiting to be made. Why not by you? Notes[1] I don't think you could give a precise definition of what counts as great work. Doing great work means doing something important so well that you expand people's ideas of what's possible. But there's no threshold for importance. It's a matter of degree, and often hard to judge at the

  Query: how to do great work?

  Helpful Answer:


In [5]:
contexts = pg_bot.retrieve_from_database(
    "how to do great work?", config=LlmConfig(number_documents=2)
)

Markdown("\n\n".join(contexts))

interesting, which you necessarily will if you're on the right path, the work will probably feel less burdensome than a lot of your peers'.The discoveries are out there, waiting to be made. Why not by you? Notes[1] I don't think you could give a precise definition of what counts as great work. Doing great work means doing something important so well that you expand people's ideas of what's possible. But there's no threshold for importance. It's a matter of degree, and often hard to judge at the

you must be interested in doing great work. And if so you're already further along than you might realize, because the set of people willing to want to is small.The factors in doing great work are factors in the literal, mathematical sense, and they are: ability, interest, effort, and luck. Luck by definition you can't do anything about, so we can ignore that. And we can assume effort, if you do in fact want to do great work. So the problem boils down to ability and interest. Can you find a

In [6]:
import fastrepl
from datasets import Dataset, load_dataset

eval = fastrepl.RAGEvaluator(node=fastrepl.RAGAS(metric="AnswerRelevancy"))
ds = load_dataset("repllabs/questions_how_to_do_great_work", split="processed")

ds = ds.shuffle(seed=8)
ds = ds.select(range(4))
ds.to_pandas()[:1]

,question,model
0,Why is originality in choosing problems consid...,gpt-3.5-turbo


In [7]:
answers = []
for q in ds["question"]:
    answer = pg_bot.query(q)
    answers.append(answer)

ds = ds.add_column("answer", answers)
ds

Dataset({
    features: ['question', 'model', 'answer'],
    num_rows: 4
})

In [8]:
result = fastrepl.local_runner(evaluator=eval, dataset=ds).run(show_progress=False)
result

Dataset({
    features: ['question', 'model', 'answer', 'result'],
    num_rows: 4
})

In [9]:
result["result"]

[0.9305319965767472,
 0.9582303203007904,
 0.9945522867737369,
 0.9199520412420689]